# Choix du pas de temps

## Pas fixe

On commence par coder une fonction qui ressoud l'équation différentielle $\dot{x} = f(x)$ avec le schéma d'Euler explicite.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_euler_explicit(f, x0, dt, n = 100):
    t = np.linspace(0, dt*(n-1), n)
    x = np.zeros(n)
    x[0] = x0
    for j in range(1, n):
        x[j] = x[j-1] + dt*f(x[j-1])
    return t, x

### Exemple avec $f : x \mapsto x$

In [ ]:
def exp_solver(solver, dt, tf = 5, x0 = 1):
    for delta_t in dt :
        t, x = solver(lambda x : x, x0, delta_t, int(tf//delta_t))
        plt.plot(t, x, label = f'dt = {delta_t}')
    plt.plot(t, [np.exp(i) for i in t], label = 'exp')
    plt.legend()
    plt.show()

In [ ]:
exp_solver(solve_euler_explicit, [0.1, 0.01])

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = solve_euler_explicit(lambda x : x, 1, dt, int(5//dt))
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/dt)}.')
print('On en déduit que le schéma est bien convergeant à l\'ordre 1 avec c_v = 400.')


On choisit maintenant un schéma à l'ordre 2 : la méthode de Heun.

In [ ]:
def heun(f, x0, dt, n = 100):
    t = np.linspace(0, dt*(n-1), n)
    x = np.zeros(n)
    x[0] = x0
    for j in range(1, n):
        x[j] = x[j-1] + dt/2*( f(x[j-1]) + f(x[j-1] + dt*f(x[j-1])) )
    return t, x

In [ ]:
exp_solver(heun, [0.1, 0.01])

Ce schéma est plus précis que celui d'Euler explicite. 

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = heun(lambda x : x, 1, dt, int(5//dt))
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/(dt**2))}.')
print('On en déduit que le schéma de Heun est convergeant à l\'ordre 2 avec c_v = 125.')

## Adaptation du pas de temps

On définit $\tilde{x}$ comme étant la vraie solution initialisée par $x^j$ au temps $t_j$. On a ainsi : $ \forall t \in [ t_0, t_f], \tilde{x}(t) = x^j + \int_{t_j}^{t} f(s,\tilde{x}(s))ds$.

On a : 

$e^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,\tilde{x}(s))ds - x^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,\tilde{x}(s))ds - ( x^{j} + (t_{j+1} - t_{j})*f(t_j,x^j))$

Soit : $e^{j+1} = \int_{t_j}^{t_{j+1}} (f(s,\tilde{x}(s)) - f(t_j,x^j))ds$

Or si $f$ est $C^1$, on a, en faisant un développement à l'ordre 1 de $s \mapsto f(s,\tilde{x}(s))$ qui est elle-même $C^1$: $ \forall s \in [ t_j , t_{j+1} ],  f(s,\tilde{x}(s)) = f(t_j,\tilde{x}(t_j)) + [\frac{\partial f}{\partial x}(t_j,\tilde{x}(t_j))+ \tilde{x'}(t_j)\frac{\partial f}{\partial y}(t_j,\tilde{x}(t_j))]  \times (s - t_j) + O((s- t_j)^2) $

Or $\tilde{x}'(t_j) = f(t_j,\tilde{x}(t_j)) = f(t_j, x^j)$ donc, on a : $\forall s \in [ t_j , t_{j+1} ],  f(s,\tilde{x}(s)) = f(t_j,x^j) + [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]  \times (s - t_j) + O((s- t_j)^2)$ 

Puis : $e^{j+1} = \int_{t_j}^{t_{j+1}} [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]   \times (s - t_j) + O((s- t_j)^2) ds = [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]\times \frac{\Delta t_j^2}{2} + O(\Delta t_j^3)$

Par ailleurs, on peut faire un faire un développement à l'ordre 1 de f en $(t_j,x(t_j))$ car f est $C^1$: $f(t_{j+1},x^{j+1}) = f(t_j+ \Delta t_j, x^j + \Delta t_j f(t_j,x^j)) = f(t_j, x^j) + \Delta t_j \frac{\partial f}{\partial x}(t_j, x^j) + \Delta t_j f(t_j,x^j) \frac{\partial f}{\partial y}(t_j, x^j) + O(||(\Delta t_j, \Delta t_j f(t_j, x^j))||^2)$

f est continue donc bornée sur ??? donc : $O(||(\Delta t_j, \Delta t_j f(t_j, x^j))||^2) = O({\Delta t_j}^2)$

Ainsi : $[\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)] = \frac{f(t_{j+1},x^{j+1}) - f(t_j, x^j) + O((\Delta t_j)^2)}{\Delta t_j}$
En combinant les deux derniers calculs, on a : $e^{j+1} = [\frac{f(t_{j+1},x^{j+1}) - f(t_j, x^j) + O((\Delta t_j)^2)}{\Delta t_j}] \times \frac{\Delta t_j^2}{2} + O(\Delta t_j^3)$